In [ ]:
import os

# go to root directory if needed
print(f"Current working directory: {os.getcwd()}")
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")
    print(f"Changed working directory to: {os.getcwd()}")


In [ ]:
import torch
import mlflow
import dagshub
import torchmetrics
import torch.nn as nn
import torch.optim as optim
import src.utils.data as data
import pytorch_lightning as pl
import torchvision.models as models

from torchvision import transforms
from sklearn.metrics import classification_report
from torchmetrics.classification import BinaryPrecision, BinaryRecall


In [ ]:
class ResNet50BinaryClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.resnet50 = models.resnet50(pretrained=False)
        self.resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, 1)

        # Initialisierung der Metriken
        self.precision = BinaryPrecision(threshold=0.5)
        self.recall = BinaryRecall(threshold=0.5)

        # Initialisieren der Listen
        self.test_labels = []
        self.test_preds = []
        self.val_outputs = []
        self.val_labels = []
        self.val_preds = []

    def forward(self, x):
        return self.resnet50(x)

    def __step(self, batch):
        images, _, labels_tuple, _ = batch

        binary_labels = [0 if label == "no_burst" else 1 for label in labels_tuple]
        binary_labels = torch.tensor(binary_labels).float().view(-1, 1)
        binary_labels = binary_labels.to(images.device)

        outputs = self(images)
        return outputs, binary_labels

    def training_step(self, batch, batch_idx):
        outputs, binary_labels = self.__step(batch)
        loss = nn.BCEWithLogitsLoss()(outputs, binary_labels)

        mlflow.log_metric("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        outputs, binary_labels = self.__step(batch)
        loss = nn.BCEWithLogitsLoss()(outputs, binary_labels)

        # Labels und Vorhersagen für spätere Verwendung speichern
        self.test_labels.append(binary_labels)
        self.test_preds.append(outputs)

        # Berechnen und protokollieren des Verlusts
        mlflow.log_metric(
            "test_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def on_test_epoch_end(self):
        # Konvertieren der gesammelten Daten in einzelne Tensoren
        test_labels = torch.cat(self.test_labels, dim=0)
        test_preds = torch.cat(self.test_preds, dim=0)

        # Berechnen der Metriken
        precision = self.precision(test_preds, test_labels)
        recall = self.recall(test_preds, test_labels)

        # Protokollieren der Metriken
        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)

        # Bereinigen der Listen für die nächste Epoche
        self.test_labels = []
        self.test_preds = []

    def validation_step(self, batch, batch_idx):
        outputs, binary_labels = self.__step(batch)

        # Labels und Vorhersagen für die spätere Verwendung speichern
        predictions = (outputs >= 0.5).int()
        self.val_labels.append(binary_labels.int())
        self.val_preds.append(predictions)

    def on_validation_epoch_end(self):
        # Alle Validierungsdaten wurden gesammelt, und wir sind am Ende der Epoche.
        val_labels = torch.cat(self.val_labels, dim=0)
        val_preds = torch.cat(self.val_preds, dim=0)

        # Berechnen der Metriken
        precision = self.precision(val_preds, val_labels)
        recall = self.recall(val_preds, val_labels)

        # Protokollieren der Metriken
        mlflow.log_metric("val_precision", precision)
        mlflow.log_metric("val_recall", recall)

        # Umwandeln in Listen für sklearn
        val_labels_list = val_labels.cpu().numpy().tolist()
        val_preds_list = val_preds.cpu().numpy().tolist()

        # Erstellen des Klassifikationsberichts
        report = classification_report(
            val_labels_list, val_preds_list, target_names=["no_burst", "burst"]
        )
        print("\nClassification Report:\n", report)

        # Vergessen Sie nicht, die Listen für die nächste Validierungsrunde zu leeren
        self.val_labels = []
        self.val_preds = []

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)

In [ ]:
# Erstellen Sie eine Instanz des Modells
model = ResNet50BinaryClassifier()
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=32,
    num_workers=0,
    val_ratio=0.15,
    test_ratio=0.15,
)
data_module.setup()

dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()

# Erstellen Sie einen Trainer für das Training
trainer = pl.Trainer(max_epochs=20)

# Starten Sie das Training
trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

# Starten Sie die Tests
trainer.test(model, dataloaders=data_module.test_dataloader())

# Speichern Sie das Modell
torch.save(model.state_dict(), "models/ResNet50BinaryClassifier.pth")
mlflow.log_artifact("models/ResNet50BinaryClassifier.pth")

# Beenden Sie die MLflow-Sitzung
mlflow.end_run()